In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style()


%matplotlib inline

In [ ]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 3000

In [ ]:
# clean = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 0)
clean1 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 1)
clean2 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 2)
clean3 = pd.read_excel('../data/CLEANED - DMC - Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_022822.xlsx', 3)
# clean4 = pd.read_excel('../data/Mental_Calls_by_Precinct_Plain_Text_Incident_Types_070121_033122.xlsx')

In [ ]:
clean1.info()

In [ ]:
clean1 = clean1.dropna()
clean1.info()

In [ ]:
clean1.Precinct.value_counts()

In [ ]:
clean2.info()

In [ ]:
clean3.info()

In [ ]:
conn = sql.connect('../data/census.sqlite')
cur = conn.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

In [ ]:
available_tables = (cur.fetchall())

In [ ]:
print(available_tables)

In [ ]:
cur.execute("SELECT * FROM B01001 LIMIT 5;")

In [ ]:
B01001 = (cur.fetchall())

In [ ]:
type(B01001)

In [ ]:
B01001

In [ ]:
pop_age_sex_df = pd.read_sql("SELECT * FROM B01001;", conn)

In [ ]:
pop_age_sex_df

In [ ]:
cur.execute("SELECT * FROM B03002 LIMIT 5;")
B03002 = (cur.fetchall())
pop_race_ethnicity_df = pd.read_sql("SELECT * FROM B03002;", conn)
pop_race_ethnicity_df

In [ ]:
cur.execute("SELECT * FROM B11001 LIMIT 5;")
B11001 = (cur.fetchall())
households_type_df = pd.read_sql("SELECT * FROM B11001;", conn)
households_type_df

In [ ]:
cur.execute("SELECT * FROM B11002 LIMIT 5;")
B11002 = (cur.fetchall())
pop_household_type_df = pd.read_sql("SELECT * FROM B11002;", conn)
pop_household_type_df

In [ ]:
cur.execute("SELECT * FROM B15002 LIMIT 5;")
B15002 = (cur.fetchall())
sex_educational_attainment_pop_25_up_df = pd.read_sql("SELECT * FROM B15002;", conn)
sex_educational_attainment_pop_25_up_df

In [ ]:
cur.execute("SELECT * FROM B19001 LIMIT 5;")
B19001 = (cur.fetchall())
household_income_df = pd.read_sql("SELECT * FROM B19001;", conn)
household_income_df

In [ ]:
cur.execute("SELECT * FROM B19013 LIMIT 5;")
B19013 = (cur.fetchall())
median_household_income_df = pd.read_sql("SELECT * FROM B19013;", conn)
median_household_income_df

In [ ]:
cur.execute("SELECT * FROM B19301 LIMIT 5;")
B19301 = (cur.fetchall())
per_capita_income_df = pd.read_sql("SELECT * FROM B19301;", conn)
per_capita_income_df

In [ ]:
cur.execute("SELECT * FROM B25024 LIMIT 5;")
B25024 = (cur.fetchall())
types_structure_df = pd.read_sql("SELECT * FROM B25024;", conn)
types_structure_df

In [ ]:
cur.execute("SELECT * FROM B25075 LIMIT 5;")
B25075 = (cur.fetchall())
value_owner_occupied_housing_units_df = pd.read_sql("SELECT * FROM B25075;", conn)
value_owner_occupied_housing_units_df

In [ ]:
cur.execute("SELECT * FROM S1701 LIMIT 5;")
S1701 = (cur.fetchall())
poverty_status_last_12_months_df = pd.read_sql("SELECT * FROM S1701;", conn)
poverty_status_last_12_months_df

In [ ]:
clean1

In [ ]:
clean1['Incident Type'].value_counts()

In [ ]:
clean1['Precinct'].value_counts()

In [ ]:
clean1['Priority'].value_counts()

In [ ]:
h = clean1.loc[clean1['Precinct'] == 'H']
h

In [ ]:
h.value_counts('Incident Type')

In [ ]:
n = clean1.loc[clean1['Precinct'] == 'N']
n

In [ ]:
n.value_counts('Incident Type')

In [ ]:
c = clean1.loc[clean1['Precinct'] == 'C']
c

In [ ]:
c.value_counts('Incident Type')

In [ ]:
s = clean1.loc[clean1['Precinct'] == 'S']
s

In [ ]:
s.value_counts('Incident Type')

In [ ]:
mt = clean1.loc[clean1['Precinct'] == 'MT']
mt

In [ ]:
mt.value_counts('Incident Type')

In [ ]:
m = clean1.loc[clean1['Precinct'] == 'M']
m

In [ ]:
m.value_counts('Incident Type')

In [ ]:
e = clean1.loc[clean1['Precinct'] == 'E']
e

In [ ]:
e.value_counts('Incident Type')

In [ ]:
w = clean1.loc[clean1['Precinct'] == 'W']
w

In [ ]:
w.value_counts('Incident Type')

In [ ]:
clean1.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
h.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
n.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
c.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
s.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
mt.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
m.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
e.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
w.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
cp = clean1.loc[(clean1['Precinct'] == 'N') 
           | (clean1['Precinct'] == 'H')]

In [ ]:
pp = clean1.loc[(clean1['Precinct'] == 'C') 
           | (clean1['Precinct'] == 'S') 
           | (clean1['Precinct'] == 'M') 
           | (clean1['Precinct'] == 'MT') 
           | (clean1['Precinct'] == 'E') 
           | (clean1['Precinct'] == 'W')]

In [ ]:
cp

In [ ]:
pp

In [ ]:
cp.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
pp.value_counts('Incident Type').plot(kind = 'bar')

In [ ]:
clean1

In [ ]:
clean1['Incident Type']

In [ ]:
clean1['Precinct']

In [ ]:
c = clean1.loc[clean1['Precinct'] == 'C']
c

In [ ]:
c_suicidal = c.loc[c['Incident Type'] == 'SUICIDAL PERSON']
c_suicidal

In [ ]:
c['Incident Type']

In [ ]:
c['Precinct']

In [ ]:
c_filtered = clean1.loc[(clean1['Precinct'] == 'C') & (clean1['Incident Type'] == 'SUICIDAL PERSON')]
c_filtered

In [ ]:
c_filtered['Created']

In [ ]:
c_filtered.value_counts('Created')

In [ ]:
c_filtered['Created'] = pd.to_timedelta(c_filtered['Created'])

In [ ]:
c_filtered['Created']

In [ ]:
c_filtered

# Created take the time delta and broadly determine the difference in volume of calls between am and pm and then again by month 


In [ ]:
july = c_filtered.loc[c_filtered['Month'] == 'July']
july

In [ ]:
july.info()

# Create 2 df that seperate police shifts i.e include everything between 10am or 10:00:00 - 22pm or 21:59:59 then another dataframe that excludes everything between the previously given times and includes everything from the start of the day and end of the day 

In [ ]:
july1 = july[july['Created'].between('0 days 10:00:00', '0 days 21:59:59')]
july1

In [ ]:
july1.info()

# 29 suicide calls between 10 am and 10 pm in the month of july for precinct c

In [ ]:
july2 = july[~ july['Created'].between('0 days 10:00:00', '0 days 21:59:59')]
july2

In [ ]:
july2.info()

In [ ]:
clean1

In [ ]:
suicide_all = clean1.loc[clean1['Incident Type'] == 'SUICIDAL PERSON']
suicide_all

In [ ]:
suicide_all.info()

In [ ]:
suicide_all['Precinct'].value_counts()

In [ ]:
suicide_all.to_csv('suicide_all.csv', index = False)

In [ ]:
suicide_all_filtered1 = suicide_all[suicide_all['Created'].between('10:00:00', '21:59:59')]
suicide_all_filtered1

In [ ]:
suicide_all_filtered1.info()

In [ ]:
suicide_all_filtered2 = suicide_all[~suicide_all['Created'].between('10:00:00', '21:59:59')]
suicide_all_filtered2

In [ ]:
suicide_all_filtered2.info()

In [ ]:
suicide_h1 = suicide_all_filtered1.loc[(suicide_all_filtered1['Precinct'] =='H')]

In [ ]:
suicide_h1.info()

In [ ]:
suicide_h2 = suicide_all_filtered2.loc[(suicide_all_filtered2['Precinct'] =='H')]

In [ ]:
suicide_h2.info()